# Load Data and Final Clean

In [119]:
df <- read.csv(file="data_010820.csv",sep=",",header=TRUE)

In [120]:
df[df$Prep.Level == 0,"Prep.Level"] = "Bachelors Prepared"

## Seperate by School

Different Schools

In [121]:
levels(factor(df$College))

[1] "GD CCPA"        "GD Education"   "GD Grad School" "GD Harpur"     
[5] "GD Management"  "GD Nursing"     "GD Watson"

In [122]:
Harpuer_df = df[df$College %in% c("GD CCPA", "GD Education","GD Grad School","GD Harpur"),]
Management_df = df[which(df$College=="GD Management"),]
Nursing_df = df[which(df$College=="GD Nursing"),]
Watson_df = df[which(df$College=="GD Watson"),]

In [123]:
H_n = dim(Harpuer_df)[1]
H_1 = sum(Harpuer_df$Y)
H_0 = H_n - H_1
M_n = dim(Management_df)[1]
M_1 = sum(Management_df$Y)
M_0 = M_n - M_1
N_n = dim(Nursing_df)[1]
N_1 = sum(Nursing_df$Y)
N_0 = N_n - N_1
W_n = dim(Watson_df)[1]
W_1 = sum(Watson_df$Y)
W_0 = W_n - W_1
School = c("Harpur","SOM","Nursing","Watson")
sch_total = c(H_n,M_n,N_n,W_n)
succ_total = c(H_1,M_1,N_1,W_1)
unsucc_total = c(H_0,M_0,N_0,W_0)
data.frame("School"=School,"Total"=sch_total,"Successful Graduates"=succ_total,"Unsuccessful Graduates"=unsucc_total)

School,Total,Successful.Graduates,Unsuccessful.Graduates
Harpur,1780,1077,703
SOM,58,38,20
Nursing,117,72,45
Watson,515,343,172


## Harpur Analysis

In [124]:
colnames(Harpuer_df)

[1] "X"                          "X1st.Term.Funding"         
 [3] "X2nd.Term.Funding"          "X3rd.Term.Funding"         
 [5] "X4th.Term.Funding"          "Start.age"                 
 [7] "College"                    "Current.Age"               
 [9] "Degree"                     "Gender"                    
[11] "Grad.Funding.Indicator"     "ID"                        
[13] "Citizenship"                "Major.ID"                  
[15] "Major"                      "Net.Cost.After.Any.Funding"
[17] "Program"                    "Prep.Level"                
[19] "Total.Charges"              "Total.Funding.Amount"      
[21] "Y"                          "PC1"                       
[23] "PC2"                        "PC3"                       
[25] "Term.1.GPA"                 "Term.2.GPA"                
[27] "Term.3.GPA"                 "Term.4.GPA"                
[29] "Term.5.GPA"                 "Term.6.GPA"                
[31] "Term.7.GPA"                 "Term.8.GPA"                
[33] "Gap"                        "Slope"                     
[35] "Valid.term"                 "Average"                   
[37] "Academic.Period..Years."

Won't use Degree column since it does not accurately show the student's major

In [125]:
levels(factor(Harpuer_df$Major))

[1] "Anthropology"                   "Art History"                   
 [3] "Art History Combined MA PhD"    "Behavioral Neuroscience"       
 [5] "Biological Sciences"            "Biological Sciences EEB"       
 [7] "Chemistry"                      "Clinical Psychology"           
 [9] "Cognitive Psychology"           "Community and Public Affairs"  
[11] "Comparative Literature"         "Economics"                     
[13] "Educational Theory and Practic" "English"                       
[15] "Geology"                        "History"                       
[17] "Material Sci and Eng-Engineerg" "Materials Science and Engineer"
[19] "Materials Science and Engr"     "Mathematics"                   
[21] "Mathematics/Economics"          "Mechanical Engineering"        
[23] "Philosophy Interpretation and"  "Philosophy SPEL"               
[25] "Physics"                        "Political Science"             
[27] "Sociology"                      "Translation Studies"           
[29] "X Geology"

In [126]:
levels(factor(Harpuer_df$Degree))

[1] "Doctor of Education"  "Doctor of Philosophy"

In [130]:
log_mod_h = glm(Y~Start.age+factor(Gender)+factor(Grad.Funding.Indicator)+Citizenship+
                Net.Cost.After.Any.Funding+factor(Prep.Level)+PC1+PC2+PC3+factor(Gap)+Slope,
                data=Harpuer_df,family="binomial")
summary(log_mod_h)


Call:
glm(formula = Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Citizenship + Net.Cost.After.Any.Funding + factor(Prep.Level) + 
    PC1 + PC2 + PC3 + factor(Gap) + Slope, family = "binomial", 
    data = Harpuer_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.6052  -0.9079   0.0001   0.9060   2.1898  

Coefficients:
                                                Estimate Std. Error z value
(Intercept)                                   -1.395e-01  4.876e-01  -0.286
Start.age                                     -1.329e-02  8.903e-03  -1.492
factor(Gender)Male                            -5.920e-02  1.181e-01  -0.501
factor(Grad.Funding.Indicator)1                9.000e-01  1.511e-01   5.956
CitizenshipAmerican Indian or Alaskan Native  -1.071e+00  9.378e-01  -1.142
CitizenshipAsia                               -2.011e-01  3.925e-01  -0.512
CitizenshipAsian                              -8.202e-01  4.626e-01  -1.773
CitizenshipBlack o

(Intercept) 
                                -1.394850e-01 
                                    Start.age 
                                -1.328554e-02 
                           factor(Gender)Male 
                                -5.920407e-02 
              factor(Grad.Funding.Indicator)1 
                                 9.000167e-01 
 CitizenshipAmerican Indian or Alaskan Native 
                                -1.071252e+00 
                              CitizenshipAsia 
                                -2.011224e-01 
                             CitizenshipAsian 
                                -8.202317e-01 
         CitizenshipBlack or African American 
                                -3.287231e-01 
                             CitizenshipChina 
                                 3.102248e-01 
                            CitizenshipEurope 
                                -6.284361e-01 
CitizenshipHawaiian or Other Pacific Islander 
                                -1.812088e+01 
                CitizenshipHispanic or Latino 
                                -1.332011e+00 
                             CitizenshipIndia 
                                 2.708264e-01 
                     CitizenshipNorth America 
                                -4.909912e-01 
                     CitizenshipSouth America 
                                -8.071986e-01 
                           CitizenshipUnknown 
                                -4.203922e-01 
                             CitizenshipWhite 
                                -4.157204e-01 
                   Net.Cost.After.Any.Funding 
                                 5.106675e-06 
           factor(Prep.Level)Masters Prepared 
                                 1.894851e+01 
                                          PC1 
                                -8.257777e-02 
                                          PC2 
                                -3.743617e-01 
                                          PC3 
                                -4.547364e-01 
                                 factor(Gap)1 
                                 8.707987e-01 
                                        Slope 
                                 2.305206e+00

In [144]:
round(summary(log_mod_h)$coeff[,c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-0.13949,0.77483
Start.age,-0.01329,0.13564
factor(Gender)Male,-0.05920,0.61628
factor(Grad.Funding.Indicator)1,0.90002,0.00000
CitizenshipAmerican Indian or Alaskan Native,-1.07125,0.25332
CitizenshipAsia,-0.20112,0.60837
CitizenshipAsian,-0.82023,0.07621
CitizenshipBlack or African American,-0.32872,0.49612
CitizenshipChina,0.31022,0.42567
CitizenshipEurope,-0.62844,0.20540


<b> Significant Variables: Funding, Citizenship-Hispanic, Net Cost, PC1,PC2,Gap,Slope

#